In [27]:
from dotenv import load_dotenv
import os

load_dotenv('./.env', override=True)

def_vars = ['DATABASE_USER', 'DATABASE_PASS', 'DATABASE_HOST', 'DATABASE_PORT', 'DATABASE_NAME']
def get_env_object():
    return { x: os.environ[x] for x in def_vars }

env_object = get_env_object()

In [81]:
import yaml


def read_file(fp: str) -> str:
    with open(fp, "r+", encoding="UTF-8") as o:
        return o.read()


def read_yaml(fp: str):
    return yaml.safe_load(read_file(fp))

def save_dict_as_file(fp:str, d:dict):
    with open(fp, "w") as y:
        yaml.dump(d, y)


In [ ]:
import psycopg2
from psycopg2 import sql


class DbConnectionManager:
    def __init__(self, env_object ,init_sql_folder: str = None):
        self.e_o = env_object
        self.con = None
        self.cur = None
        
        if init_sql_folder:
            print("will create default objects")
            self.init_db(init_sql_folder)
            print("successfully created")
    
    def _call_func(self, func_name: str, params: list[str], **kwargs):

        commit = kwargs.get("commit") or False
        res = kwargs.get("ret_result") or False

        q = sql.SQL("""
            select {func_name}({params_list});
        """).format(
            func_name=sql.Identifier(func_name),
            params_list= sql.SQL(', ').join(
                [sql.Literal(x) for x in params])
        )

        return self._execute_query(q, commit=commit, ret_result=res)
    
    def try_save_query(self, view_name:str, query: str) -> bool:
        ret = self._call_func(
            'f_save_query_as_view',
            [view_name, query],
            commit=True,
            ret_result=True
        )
        return ret[0][0]
    
    def check_generated_query(self, corr_view: str, tested_view: str) -> dict:
        ret = self._call_func(
            'f_check_generated_query',
            [corr_view, tested_view],
            ret_result=True
        )
        return ret[0][0]
    

    def init_db(self, init_sql_folder: str) -> None:
        if not os.path.exists(init_sql_folder):
            raise Exception("init sql folder does not exists")
        
        for f in os.listdir(init_sql_folder):
            fp = os.path.abspath(
                os.path.join(init_sql_folder, f)
            )

            fc = read_file(fp)
            
            self._execute_query(fc, commit=True)


    def _execute_query(self, query:str, commit=False, ret_result=False):
        cur, con = self._get_connection()

        cq = query
        if isinstance(query, sql.Composed):
            cq = query.as_string(cur)

        try:
            cur.execute(cq)
        except Exception as e:
            print(e)
            return None
        
        if commit:
            con.commit()

        if ret_result:
            return cur.fetchall()
        
        return None

    def _is_con_set(self):
        return self.cur is not None and self.con is not None
        
    def _get_connection(self):
        if self._is_con_set():
            return (self.cur, self.con)
        
        con_str = self._get_con_string()
        self.con = psycopg2.connect(con_str)
        self.cur = self.con.cursor()
        return (self.cur, self.con)
    
    def __del__(self):
        if self._is_con_set():
            self.cur.close()
            self.con.close()
        
    
    def _get_con_string(self):
        return "host='%s' dbname='%s'\
            user='%s' password='%s'" % (
                self.e_o["DATABASE_HOST"],
                self.e_o["DATABASE_NAME"],
                self.e_o["DATABASE_USER"],
                self.e_o["DATABASE_PASS"]
            )



In [ ]:
import abc

class LLMProvider(metaclass=abc.ABCMeta):

    @staticmethod
    def __subclasshook__(cls, subclass):
        return (
                hasattr(subclass, "generate_sql") and 
                callable(subclass, "generate_sql") and 

                hasattr(subclass, "get_service_name") and 
                callable(subclass, "get_service_name") or
                
                NotImplemented
        )

    def generate_sql(self, user_query: str) -> str:
        """ Generate sql. Prompt should be known to class """
        raise NotImplementedError
    
    def get_service_name(self) -> str:
        """ Return service name. 
            E.g for Lamini LLM, will return lamini
        """
        raise NotImplementedError

In [89]:
class LLMResultComparator:
    def __init__(
            self,
            data_folder: str,
            db_manager: DbConnectionManager):
        
        self.db_manager = db_manager
        self.df = data_folder
        self._init_db_with_cases()

    
    def compute_accuracy(self, llm: LLMProvider):

        gen_q = {}
        tc: dict = self.tc
        sn = llm.get_service_name()

        print("start getting generated queries")
        for x in tc:
            data: dict = tc.get(x)
            gen_q[x] = llm.generate_sql(data.get("user_query"))
        
        # на всякий случай
        print("dump generated queries to file")
        fn = f"{self.df}/{sn}.yaml"
        save_dict_as_file(fn, gen_q)

        stats = {}
        for x in tc:
            vn = self._get_gen_view_name(x, sn)
            sr = self.db_manager.try_save_query(
                vn,
                gen_q.get(x)
            )

            if not sr:
                stats[x] = {
                    "status": "failed",
                    "reason": "query not executable"
                }
                continue
            
            comp_res = self.db_manager.check_generated_query(
                self._get_correct_view_name(x),
                vn
            )

            res = self._analyze_query_comp(comp_res)
            stats[x] = {
                "status": res["status"],
                "reason": res["description"]
            }

        return stats

    def _analyze_query_comp(self, comp_res: dict) -> dict:
        res = {
            "status":"",
            "description":""
        }

        acm: int= comp_res.get("attr_count_mismatch")
        rcm: int = comp_res.get("rows_count_mismatch")
        chm: list[str] = comp_res.get("cols_hashes_mismatch")

        if (acm == 0 and rcm == 0 and chm == []):
            res["status"] = "success"
        else:
            res["status"] = "failed"
            desc = []

            if acm != 0:
                desc.append("attribute count mismatch")
            if rcm != 0:
                desc.append("rows count mismatch")
            if chm != []:
                desc.append("execution results differs")
            
            res["description"] = ';'.join(desc)

        return res


    def _get_correct_view_name(self, test_case: str) -> str:
        return f"v_{test_case}"
    
    def _get_gen_view_name(self, test_case: str, service_name: str) -> str:
        return f"{service_name}_{test_case}"

    def _init_db_with_cases(self):
        tc = read_yaml(f"{self.df}/main.yaml")["test_cases"]

        # если будет просадка по скорости не забыть добавить пред проверку
        # на инит базы
        for x in tc:
            data = tc.get(x)
            fn = self._get_correct_view_name(x)
            res = self.db_manager.try_save_query(fn, data.get("correct_query"))
            if not res:
                print(f"failed to save {fn}")
        
        self.tc = tc





In [85]:
class DummyLLM(LLMProvider):
    """
    Class only for test and concept purpose
    """

    def __init__(self, context_manager):
        self.pr = context_manager

    def generate_sql(self, user_query: str) -> str:
        if 'видеоигры' in user_query:
            return 'select id from video_games.game'
        else:
            return 'select * from video_games.genre'
    
    def get_service_name(self) -> str:
        return "dummy"

In [86]:
class DefaultContextManager:
    def __init__(self, m_folder: str):
        if not os.path.exists(m_folder):
            raise Exception("missing file with context")

        self.context = read_file(f"{m_folder}/default.txt")
    
    def context(self):
        return self.context


In [59]:
db_manager = DbConnectionManager(env_object, './init_sql')

will create default objects
successfully created


In [90]:
f = LLMResultComparator("test_data", db_manager)

In [91]:
test_res = f.compute_accuracy(DummyLLM(DefaultContextManager("./contexts")))

start getting generated queries
dump generated queries to file


In [92]:
test_res

{'test_1': {'status': 'failed',
  'reason': 'attribute count mismatch;execution results differs'},
 'test_2': {'status': 'success', 'reason': ''}}

In [61]:
res1 = db_manager.check_generated_query("test_001", "test_1102")

In [82]:
res1

{'attr_count_mismatch': 0,
 'rows_count_mismatch': -53960,
 'cols_hashes_mismatch': ['25265c47e5182330fe2e74235c5a1e75',
  '38654549aaf1f3e0eb259e9cc3902cfd',
  '098c1ebc8d18a24e4958e06122877b86']}